# Svenskt Kvinnobiografiskt lexikon part 1
version 1.3.4.31 


* Denna [Jupityr Notebook](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon.ipynb) 
  * [del 2](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%202.ipynb)
* Wikidata property [P4963](https://www.wikidata.org/wiki/Property_talk:P4963) 


# Check diff SKBL Wikidata 
ett försök att hålla Wikidata / SKBL i fas 
* [KARP APIet](https://spraakbanken.gu.se/verktyg/karp/webb-api)
* se tidigare försök att ha ISNI i synk [T219700](https://phabricator.wikimedia.org/T219700), GITHUB [salgo60/SKBLWikidata](https://github.com/salgo60/SKBLWikidata) 
  * mar 2019 kommentar om [platser i SKBL](https://phabricator.wikimedia.org/T218782)  
  
Viss osäkerhet hur en bra API fråga skall se ut returnerar alla personer i SKBL

In [139]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None)
pd.set_option('display.max_rows', None)

dftot = pd.DataFrame()
 
url = "https://ws.spraakbanken.gu.se/ws/karp/v5/minientry?q=extended%7C%7Cand%7Cnamn%7Cexists&mode=skbllinks&show=name%2Curl&sort=sorteringsnamn.sort%2Csorteringsnamn.init%2Ctilltalsnamn.sort&size=10000&resource=skbl"
r = http.request('GET', url) 
data = json.loads(r.data.decode('utf-8')) 
#print(type(data))
#print(r.info())
listSKBL = []
i = 0 
for row in (data["hits"]["hits"]):
    #print (row["_source"]["url"])
    listSKBL.append(row["_source"]["url"])
    i = i + 1 
    #print (i)
SKBLtot = pd.DataFrame(listSKBL,
                  columns=['skblid'])


In [140]:
SKBLtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 1 columns):
skblid    1520 non-null object
dtypes: object(1)
memory usage: 12.0+ KB


In [141]:
#SKBLtot

In [142]:
#SKBLtot.sort_index(inplace=True) 
SKBLtot.sort_values(['skblid'], ascending=[1],inplace=True) 

In [143]:
# The API gives us some duplicates 
SKBLtot.drop_duplicates(subset ="skblid", keep = False, inplace = True)
SKBLtot.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1391 entries, 539 to 818
Data columns (total 1 columns):
skblid    1391 non-null object
dtypes: object(1)
memory usage: 21.7+ KB


In [144]:
SKBLtot = SKBLtot.reset_index(drop=True)  
#SKBLtot

# Wikidata

In [145]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT * WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P4963 ?skbl
} order by ?skbl"""


def get_results(endpoint_url, query):
    user_agent = "User:salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)
 
lst = []
for result in results["results"]["bindings"]:
    lst.append((result["skbl"]["value"]))
WDtot = pd.DataFrame(lst,columns={'skblid'}) 
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 1 columns):
skblid    1454 non-null object
dtypes: object(1)
memory usage: 11.5+ KB


In [146]:
WDtot.sort_index(inplace=True)  

In [147]:
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 1 columns):
skblid    1454 non-null object
dtypes: object(1)
memory usage: 11.5+ KB


In [148]:
#WDtot

### Check duplicates SKBL and Wikidata

In [149]:
dfmerge = pd.merge(WDtot, SKBLtot,how='outer', indicator=True)

In [150]:
# check whatsnew / deleted  
dfmerge['_merge'].value_counts()

both          1391
left_only       63
right_only       0
Name: _merge, dtype: int64

In [151]:
#dfmerge

In [152]:
SKBLnew = dfmerge[dfmerge['_merge']=="right_only"]


In [153]:
SKBLnew

,skblid,_merge


In [154]:
SKBLnew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 2 columns):
skblid    0 non-null object
_merge    0 non-null category
dtypes: category(1), object(1)
memory usage: 104.0+ bytes


In [155]:
import csv  
jsonURL = "https://skbl.se/sv/artikel/"
listNewItems =[]
for index,row in SKBLnew.iterrows():
    url = jsonURL + row["skblid"] + ".json"
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))  
    
    new_item = dict()
    try:
        birthdate = data['lifespan']['from']['date']['date'],
    except:
        birtdate =""
    new_item['birthdate'] = birthdate
    try:
        deathdate = data['lifespan']['to']['date']['date']
    except:
        deathdate=""
    new_item['deathdate'] = deathdate
    try:
        birthparish = data['lifespan']['from']['place']['parish']
    except:
        birthparish = ""        
    new_item['birthparish'] = birthparish
    try:
        deathparish = data['lifespan']['to']['place']['parish']
    except:
        deathparish = ""
    new_item['deathparish'] = deathparish
    try:
        ISNI = data['id']['id']
    except:
        ISNI = ""
    new_item['ISNI'] = ISNI
    new_item['SKBL'] = row["skblid"]
    new_item['index'] = index
    new_item['skbl_status'] = data['skbl_status']
    new_item['firstname'] = data['name'].get('firstname')
    new_item['lastname'] = data['name'].get('lastname')
    new_item['subtitle'] = data['subtitle']
    listNewItems.append(new_item)
print (len(listNewItems) ," antal poster")
if len(listNewItems) > 0:
    keys = listNewItems[0].keys()
    with open("SKBL_newfile.csv", "w", newline='') as SKBLfile:
        dict_writer = csv.DictWriter(SKBLfile, keys)
        dict_writer.writeheader()
        dict_writer.writerows(listNewItems)
else:
    print ("Ingen fil skapas inga nya poster") 
            
    

0  antal poster
Ingen fil skapas inga nya poster


In [156]:
#dfmerge

In [157]:
def make_WDclickable(val):
    return '<a target="_blank" href="https://hub.toolforge.org/P4963:{}?site=wikidata">WD {}</a>'.format(val, val)
def make_SKBLclickable(val):
    return '<a target="_blank" href="https://skbl.se/en/article/{}">SKBL {}</a>'.format(val, val)


#dfmerge['WD'] = dfmerge.apply(lambda x: make_WDclickable(x['skblid']), axis=1)
#dfmerge['SKBL'] = dfmerge.apply(lambda x: make_SKBLclickable(x['skblid']), axis=1)
 
#dfmerge['WD'] = dfmerge['skblid']
#dfmerge['SKBL'] = dfmerge['skblid']
#dfmerge = dfmerge.style.format({'WD': make_WDclickable})  
#dfmergedfmerge.style.format({'SKBL': make_SKBLclickable})  
SKBLnew_left = dfmerge[dfmerge['_merge']=="left_only"]

#SKBLnew_left = SKBLnew_left.to_html(escape=False)
#SKBLnew_left.write(SKBLnew_left, unsafe_allow_html=True)
#SKBLnew_left